In [1]:
import pandas as pd
import html5lib
import requests
from bs4 import BeautifulSoup
import time
import urllib.request  # we are going to need to generate a Request object
from tqdm import tqdm

In [2]:
def transfermarkt_call_soup(url):
    # here we define the headers for the request
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    # this request object will integrate your URL and the headers defined above
    req = urllib.request.Request(url=url, headers=headers)

    # calling urlopen this way will automatically handle closing the request
    with urllib.request.urlopen(req) as response:
        page_html = response.read()
    return BeautifulSoup(page_html, 'html.parser')

# Scrape Teams

In [76]:
super_league = "https://www.transfermarkt.com/raiffeisen-super-league/startseite/wettbewerb/C1"
soup = transfermarkt_call_soup(super_league)

In [9]:
table = soup.find_all('table',{"class":"items"})

In [56]:
df.loc[df['name']=='BSC Young Boys','teamlink'] = 1

In [59]:
df = pd.DataFrame(pd.read_html(str(soup.find_all('table',{"class":"items"})[0]))[0])
new_columns = df.columns[2:]
new_columns = new_columns.drop('name.1')
#df = df[new_columns]
#df.drop(df.columns[0])
df = df.drop(df.columns[:1], 1)
df = df.drop(df.columns[1], 1)
df = df.drop(df.columns[-1], 1)
df.columns = new_columns
df['ø age'] = df['ø age']/10

df['teamlink'] = ''
df.loc[df['name']=='BSC Young Boys','teamlink'] = "https://www.transfermarkt.com/bsc-young-boys/kader/verein/452/"
df.loc[df['name']=='FC Basel 1893','teamlink'] = "https://www.transfermarkt.com/fc-basel-1893/startseite/verein/26/"
df.loc[df['name']=='FC Zürich','teamlink'] = "https://www.transfermarkt.com/fc-zurich/startseite/verein/260/"
df.loc[df['name']=='FC Luzern','teamlink'] = "https://www.transfermarkt.com/fc-luzern/startseite/verein/434/"
df.loc[df['name']=='FC Sion','teamlink'] = "https://www.transfermarkt.com/fc-sion/startseite/verein/321/"
df.loc[df['name']=='FC Lugano','teamlink'] = "https://www.transfermarkt.com/fc-lugano/startseite/verein/2790/"
df.loc[df['name']=='FC Thun','teamlink'] = "https://www.transfermarkt.com/fc-thun/startseite/verein/938/"
df.loc[df['name']=='FC St. Gallen 1879','teamlink'] = "https://www.transfermarkt.com/fc-st-gallen/startseite/verein/257/"
df.loc[df['name']=='Servette FC','teamlink'] = "https://www.transfermarkt.com/servette-fc/startseite/verein/61/"
df.loc[df['name']=='Neuchâtel Xamax FCS','teamlink'] = "https://www.transfermarkt.com/neuchatel-xamax-fcs/startseite/verein/9084/"

df.to_csv('../data/teams.csv', index=False)


# Scrape Players

In [46]:
teams_df = pd.read_csv('../data/teams.csv')
teams_df.drop(teams_df.tail(1).index,inplace=True)

In [48]:
data = {
    'team': [],
    'name': [],
    'id': [],
    'transfermarktlink': [],
    'birthday': [],
    #'nationality': [],
    'market_value': [],
}

for index, row in tqdm(teams_df.iterrows()):
    team_link = row['teamlink']
    team_name = row['name']
    
    soup = transfermarkt_call_soup(team_link)
    all_tr = soup.find_all('tr', {'class': ['odd', 'even']})
    for row in all_tr:
        all_td = row.find_all('td', recursive=False)

        #for column in all_td:
        #    print(' >', column.text)
        data['name'].append( all_td[1].text.split('.')[0][:-1] )
        data['id'].append( all_td[1].find("table").find("td", {'class': 'hauptlink'}).span.a['id'] )
        data['transfermarktlink'].append( all_td[1].find("table").find("td", {'class': 'hauptlink'}).span.a['href'] )
        data['birthday'].append( all_td[2].text[:-5])
        data['team'].append(team_name)
        #data['nationality'].append( all_td[3].find('img')['title'] )
        data['market_value'].append( all_td[5].text )

df = pd.DataFrame(data)


10it [00:26,  2.67s/it]


In [69]:
def market_value_parse(marketvalue):
    splitted = marketvalue.split(' ')
    splitted[0] = splitted[0].replace(',', '')
    if splitted[1] == "K":
        splitted[1] = "000"
    if splitted[1] == "mil.":
        splitted[1] = "00000"
    number_string = splitted[0] + splitted[1]
    return int(float(number_string) * 1.1) #eur to chf

In [70]:
df['marketvalue'] = df['market_value'].apply(lambda x: market_value_parse(x))

In [76]:
df.to_csv('../data/player_static.csv', index=False)

3500000.0

In [325]:
#page = requests.get(my_url)
soup = BeautifulSoup(page_html, 'html.parser')
table = soup.find_all('table',{"class":"items"})
#table_header = table[0].theader
table = table[0].tbody
player_rows = table.find_all("tr")

In [326]:
player_df = pd.DataFrame()
for player_row in player_rows[1:]:
    #parse inline_table    
    player_tags = player_row.find_all("td")
    
    player_number = player_tags[0].text
    player_name = player_tags[1].div.text #better a?
    player_link = player_tags[1].div.span.a['href']
    player_id = player_tags[1].div.span.a['id']
    try:
        player_smallimg = player_tags[2].a.img['src']
    except:
        continue
    player_birthday = player_tags[5].text
    player_country = player_tags[6].img["title"]
    player_value = player_tags[8].text

    player_dict = {
        #'player_tags'     :player_tags
        'player_number'  :player_number
        ,'player_name'    :player_name
        ,'player_link'    :player_link
        ,'player_id'      :player_id
        ,'player_smallimg':player_smallimg
        ,'player_birthday':player_birthday
        ,'player_country' :player_country
        ,'player_value'   :player_value
    }
    #for player_tag in player_tags:
    #    print(player_tag[])
    player_df = player_df.append(player_dict, ignore_index=True)
        


IndexError: list index out of range